Datasets:
Complete Binary, w/ background - sizes 1 and 10 and highly mixed 5
Complete Binary, no background - sizes 1 and 10 and incremental 5
Custom Smoothing, w/ background - sizes 1 and 10 - This is the baseline
Custom Smoothing, no background - sizes 1 and 10


Loss settings:
Custom Smoothing, enabled - Loss A -This is the baseline
Custom Smoothing, disabled - Loss A
No Smoothing, disabled - Loss A or B 
Proper Smoothing, disabled - Loss B


Model Types: 
Standard Dilation, No Attention - This is the baseline
Standard Dilation, Attention
Reduced Dilation, No Attention
Reduced Dilation, Attention

Todo: Need to make a reduced dilation version of the model - done, model takes dilation scaler
Confirm that if no smoothing and early reward disabled, Loss A and Loss B are equivalent - confirmed, so long as parameters and thresholds are the same

Checklist of datasets:

 Complete Binary, w/ background

 Size 1 - COMPLETE

 Size 10

 Highly mixed 5


 Complete Binary, no background

 Size 1

 Size 10

 Incremental 5 - have a custom smoothed version w/ background that can be converted

 Custom Smoothing, w/ background (This is the baseline)

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Custom Smoothing, no background

 Size 1

 Size 10


In [2]:
import time
import sys
import os
import glob
import math
import threading
import concurrent.futures as cf
import random
import re

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model, layers, metrics, losses, callbacks, optimizers, models, utils
from keras import backend as K
import gc
import keras_tuner as kt
from pyfaidx import Fasta

import sys
import os

datasets_path = "../../Datasets/"
models_path = "../../Models/"

2025-04-12 21:51:14.932147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-12 21:51:15.028668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-12 21:51:15.056663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-12 21:51:15.232283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-12 21:51:16.827781: W tensorflow/compiler/tf2

In [3]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
import IEModules as iem

In [6]:
iem.Genetic_Data_Pipeline.test_function("Howdy!")

You entered Howdy! into test_function


'Howdy!'

In [8]:
import inspect
functions = inspect.getmembers(iem.Genetic_Data_Pipeline, inspect.isfunction)
for name, func in functions:
    print(name)

Input
_bytes_feature
_float_feature
_int64_feature
build_chunk_data_for_indices
build_dataset_from_tfrecords
bytes_feature
calculate_introns
compute_chunk_indices
convert_and_write_tfrecord
convert_labels_to_binary
float_feature_list
int_feature_list
label_sequence_local
load_gtf_annotations
main
one_hot_encode_reference
pad_encoded_seq
pad_labels
parse_chunk_example
search_gtf_by_range
serialize_chunk_example
serialize_example_with_metadata_no_convert
split_tfrecords
stream_shuffled_records
swap_columns_if_needed
test_dataset_from_tfrecords
test_function
trim_chr_genome
tvt_split_tfrecords
write_shuffled_records_to_single_tfrecord
write_tfrecord_in_shards_hybrid
write_to_shard_with_threads


Check Genetic_Data_Pipeline for correctness

Check Custom_Models for correctness

Need to make a shard shuffler module probably that goes from output of genetic_data_pipeline to fully shuffled test-train-split

In [3]:
iem.Genetic_Data_Pipeline.dataset_pipeline_help()


          1. build_initial_shards function generates 4 tfrecord shards at each shift fed to it.
          I recommend giving a list of 1 window shift.  Currently paths are hardcoded.  
          The first 4*n characters of the shards will be the shift numbers where n is 
          the number of shifts fed to the function.
          
          2. split_tfrecords takes an input and output directory and number of file splits
          and splits each TFRecord in the input_directory (assumed to be gzipped TFRecords)
          into 'num_splits' smaller TFRecords.
          
          3. write_shuffled_records_to_single_tfrecord builds a single big, shuffled, 
          gzip-compressed TFRecord file 
          
          Args:
            input_dir (str): Directory containing the source TFRecord files.
            allowed_indices (list): List of allowed starting indices.
            output_filepath (str): Full path to the output TFRecord file.
            
          4. tvt_split_records tes

Checklist of datasets:

 Complete Binary, w/ background

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Highly mixed 5                                 Need to build highly mixed 5 and convert to binary


 Complete Binary, no background

 Size 1 - background removed when loading into model (Data_Functions.py)

 Size 10 - background removed when loading into model (Data_Functions.py)

 Incremental 5 - have a custom smoothed version w/ background that can be converted   conversion ongoing

 Custom Smoothing, w/ background (This is the baseline)

 Size 1 - COMPLETE

 Size 10 - COMPLETE

 Custom Smoothing, no background

 Size 1 - background removed when loading into model (Data_Functions.py)

 Size 10 - background removed when loading into model (Data_Functions.py)



 Size 10 has 2365261 records

In [ ]:
# iem.Genetic_Data_Pipeline.convert_and_write_tfrecord(datasets_path+"AugDataSets/all_ten_shuffled.tfrecord.gz", datasets_path+"AugDataSets/binary_ten_shuffled.tfrecord.gz")

I0000 00:00:1744095490.509284 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.699680 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.699799 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.704425 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744095490.704488 1330704 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [ ]:
# iem.Genetic_Data_Pipeline.convert_and_write_tfrecord(datasets_path+"AugDataSets/fifths_incremental_shuffled.tfrecord.gz", datasets_path+"AugDataSets/binary_fifths_incrememntal.tfrecord.gz")

In [ ]:
# iem.Genetic_Data_Pipeline.transform_tfdataset(datasets_path+"AugDataSets/AllTen", datasets_path+"AugDataSets", 45, 0.5)
# print("Done")

Temporary folder created at: ../../Datasets/AugDataSets/AllTen/temp_shards


I0000 00:00:1744246104.885834  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744246105.075605  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744246105.075745  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744246105.078746  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744246105.078828  547085 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Total records found in input dataset: 2365261
Sampling 1182630 records (fraction = 0.5).


2025-04-09 19:14:36.772563: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Split all_ten_shuffled.tfrecord.gz into 45 shards.
10000 records written so far...
20000 records written so far...
30000 records written so far...
40000 records written so far...
50000 records written so far...
60000 records written so far...
70000 records written so far...
80000 records written so far...
90000 records written so far...
100000 records written so far...
110000 records written so far...
120000 records written so far...
130000 records written so far...
140000 records written so far...
150000 records written so far...
160000 records written so far...
170000 records written so far...
180000 records written so far...
190000 records written so far...
200000 records written so far...
210000 records written so far...
220000 records written so far...
230000 records written so far...
240000 records written so far...
250000 records written so far...
260000 records written so far...
270000 records written so far...
280000 records written so far...
290000 records written so far...
3

In [ ]:
# iem.Genetic_Data_Pipeline.convert_and_write_tfrecord(datasets_path+"AugDataSets/50_all_ten_shuffled.tfrecord.gz", datasets_path+"AugDataSets/binary_5x_mixed.tfrecord.gz")

2025-04-10 04:28:15.601091: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Dataset New Naming Scheme: 
Epoch Units: 01-10, Style: M for Mixed or I for Incremental, Smoothing: C for custom smoothing, B for Binary
Epoch-Units_Style_Smoothing_list-of-shifts-if-incremental-divided-by-1000_IEData.tfrecord.gz
ex: 02_I_B_0-2.5_IEData.tfrecord.gz

Per line, this is input from trial csv:
Trial,Smoothing,Background,Early Rewarding,Size,Style,Dilation,Attention

Trial becomes folder where model data are saved

Smoothing, Size, Style define the dataset

Background is a choice handled in the data loader which returns a dataset generator pretending to be a dataset (Data_Functions.prep_dataset_from_tfrecord())

Early rewarding: if true, need an option in the loss functions or something that checks epoch number.  Rewarding==True for more than 3 epoch units is overkill so the switch should flip after 3 epoch units

Dilation and Attention are model hyperparameters

What I need then:
Experiment handler function (class?) that reads a line from the csv and trains for 10 epoch units.  Needs a working folder given to it (Experiment Name is probably the folder name) and a subfolder for each trial (Trial_01, etc).  Checkpoints get saved in there, as does the model history which gets saved to json every epoch unit but overwritten, as should the StatefulReduceLROnPlateau callback state that I have yet to implement.
The experiment should track which trial and which epoch was most recently saved and be able to resume from that trial and checkpoint easily.  Maybe have it save a jpg of the train validate curve at the end of the trial using the function in Helper_functions.py

In [5]:
data_directory = datasets_path+"Experiment 01/"
save_directory = data_directory+"TestValTrain/"
for root, dirs, files in os.walk(data_directory):
    if 'TestValTrain' in dirs:
        dirs.remove('TestValTrain')
    for file in files:
        file_path = data_directory+os.path.basename(file)
        train_save_file_path = save_directory+"Train_"+os.path.basename(file)
        val_save_file_path = save_directory+"Val_"+os.path.basename(file)
        test_save_file_path = save_directory+"Test_"+os.path.basename(file)
        iem.Genetic_Data_Pipeline.tvt_split_tfrecords(file_path, train_save_file_path, val_save_file_path,test_save_file_path)
        print(os.path.basename(file)+" split")

I0000 00:00:1744516349.562100 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744516349.758718 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744516349.758835 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744516349.762785 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744516349.762879 1396344 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Total records found: 1182630
Splitting into -> Train: 946104, Val: 118263, Test: 118263


2025-04-12 21:58:53.921334: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 22613 of 25000
2025-04-12 21:58:55.008046: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
2025-04-12 22:51:29.014481: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Final Split Counts -> Train: 946104, Val: 118263, Test: 118263
05_M_B_IEData.tfrecord.gz split
Total records found: 2365261
Splitting into -> Train: 1892208, Val: 236526, Test: 236527


2025-04-12 23:04:33.559500: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:15: Filling up shuffle buffer (this may take a while): 20358 of 25000
2025-04-12 23:04:35.240383: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


Final Split Counts -> Train: 1892208, Val: 236526, Test: 236527
10_M_C_IEData.tfrecord.gz split


2025-04-13 00:55:15.264840: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# #!/usr/bin/env python3
# """
# Experiment Handler Module

# This module defines an ExperimentHandler class that:
#   • Reads a single trial’s configuration (with keys: Trial, Smoothing, Background,
#     Early Rewarding, Size, Style, Dilation, Attention) from a CSV file.
#   • Creates an experiment folder under a common experiment root for each trial.
#   • Loads a dataset from a tfrecord using a constructed filename based on the configuration.
#   • Builds a model via a provided model builder or a default placeholder.
#   • Trains the model for 10 "epoch units" (one epoch per unit in this example).
#   • Saves checkpoints, training history (using Helper_Functions.save_history_to_json),
#     and a train/validation curve plot (using Helper_Functions.plot_train_val_curve) after each epoch unit.
#   • Supports resuming progress from a saved state.
# """


# import time
# import sys
# import os
# import glob
# import math
# import threading
# import concurrent.futures as cf
# import random
# import re
# import json

# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from keras import Input, Model, layers, metrics, losses, callbacks, optimizers, models, utils
# from keras import backend as K
# import gc
# import keras_tuner as kt
# from pyfaidx import Fasta

# import sys
# import os

# datasets_path = "../../Datasets/"
# models_path = "../../Models/"
# import IEModules as iem


# # Optionally import custom callbacks from Custom_Callbacks if available
# # from Custom_Callbacks import StatefulReduceLROnPlateau  # placeholder for stateful LR scheduling

# class ExperimentHandler:
#     def __init__(self, trial_config, experiment_root="ExperimentResults", model_builder=None):
#         """
#         Initialize experiment for a single trial.
        
#         Parameters:
#           trial_config (dict): Dictionary with keys:
#              "Trial", "Smoothing", "Background", "Early Rewarding", "Size", "Style", "Dilation", "Attention"
#           experiment_root (str): The root directory for experiment results.
#           model_builder (callable, optional): Function that builds and compiles the model.
#                Defaults to a simple placeholder model.
#         """
#         self.trial_config = trial_config
#         self.total_epoch_units = 10
#         self.current_epoch_unit = 0
#         self.model = None
#         self.model_builder = model_builder if model_builder is not None else self.default_model_builder

#         # Create a subfolder for the trial using the "Trial" field under the experiment root.
#         self.trial_folder = os.path.join(experiment_root, self.trial_config["Trial"])
#         os.makedirs(self.trial_folder, exist_ok=True)
#         self.state_file = os.path.join(self.trial_folder, "state.json")
#         self.dataset = None

#     def default_model_builder(self):
#         """
#         Build a default placeholder model.
#         Replace this with a call to your custom model builder (e.g., Custom_Models.create_dcnn_model).
#         """
#         model = models.Sequential([
#             layers.InputLayer(input_shape=(5000, 5)),
#             layers.Conv1D(32, 3, activation='relu', padding='same'),
#             layers.GlobalAveragePooling1D(),
#             layers.Dense(5, activation='sigmoid')
#         ])
#         model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["f1_score"])
#         return model

#     def load_dataset(self):
#         """
#         Constructs the tfrecord filename based on trial configuration and loads the dataset.
#         Modify the logic below to reflect your specific naming convention.
#         """
#         # For example: "Epoch-Units_Style_Smoothing_list-of-shifts-if-incremental-divided-by-1000_IEData.tfrecord.gz"
#         epoch_unit_placeholder = "XX"  # Placeholder: replace or update with epoch unit if needed.
#         style = self.trial_config["Style"]  # e.g., "M" or "I"
#         smoothing = self.trial_config["Smoothing"]  # e.g., "C" (custom) or "B" (binary)
#         # For incremental style, include shifts (placeholder here as "0-2.5")
#         shifts = "0-2.5" if style.upper() == "I" else ""
#         file_name = f"{epoch_unit_placeholder}_{style}_{smoothing}_{shifts}_IEData.tfrecord.gz"
#         # Adjust the base path for your tfrecords:
#         tfrecord_path = os.path.join("path_to_tfrecords", file_name)
        
#         # Determine if the Background should be removed (interpreting string to boolean).
#         cut_background = str(self.trial_config["Background"]).lower() in ["true", "1"]
#         # Use the "Size" field for the batch size.
#         batch_size = int(self.trial_config["Size"])
#         self.dataset = iem.Data_Functions.prep_dataset_from_tfrecord(
#             tfrecord_path,
#             batch_size=batch_size,
#             shuffled=True,
#             cut_background=cut_background
#         )
    
#     def resume_progress(self):
#         """
#         Resumes from a previously saved state if it exists.
#         Loads the current epoch unit and last saved checkpoint.
#         """
#         if os.path.exists(self.state_file):
#             with open(self.state_file, "r") as f:
#                 state = json.load(f)
#             self.current_epoch_unit = state.get("current_epoch_unit", 0)
#             last_checkpoint = state.get("last_checkpoint", None)
#             if last_checkpoint and os.path.exists(last_checkpoint):
#                 self.model = models.load_model(last_checkpoint, compile=False)
#             print(f"Resumed trial {self.trial_config['Trial']} at epoch unit {self.current_epoch_unit}")
#         else:
#             print(f"Starting new trial {self.trial_config['Trial']}")
    
#     def save_state(self, checkpoint_path):
#         """
#         Saves the current experiment state to allow resuming later.
#         """
#         state = {
#             "current_epoch_unit": self.current_epoch_unit,
#             "last_checkpoint": checkpoint_path
#         }
#         with open(self.state_file, "w") as f:
#             json.dump(state, f)
#         print(f"Saved state for trial {self.trial_config['Trial']} to {self.state_file}")

#     def train_epoch_unit(self):
#         """
#         Trains the model for one epoch unit.
#         Implements early rewarding logic (active only for the first 3 epoch units),
#         saves a checkpoint, training history, and a plot of the train/validation curve.
#         """
#         checkpoint_path = os.path.join(self.trial_folder, f"checkpoint_epochunit_{self.current_epoch_unit+1}.h5")
#         cp_callback = callbacks.ModelCheckpoint(
#             filepath=checkpoint_path,
#             save_weights_only=False,
#             save_freq="epoch",
#             verbose=1
#         )
#         callbacks_list = [cp_callback]
        
#         # Early Rewarding: Only effective if set in the config and for the first 3 epoch units.
#         early_reward = str(self.trial_config["Early Rewarding"]).lower() in ["true", "1"]
#         if early_reward and self.current_epoch_unit >= 3:
#             early_reward = False
#         print(f"Trial {self.trial_config['Trial']}: Early Rewarding set to {early_reward} at epoch unit {self.current_epoch_unit+1}")
        
#         # Train for one epoch unit. (Here, one epoch = one epoch unit.)
#         history = self.model.fit(
#             self.dataset,
#             epochs=1,
#             callbacks=callbacks_list,
#             validation_data=self.dataset,  # Adjust for a dedicated validation dataset if available.
#             verbose=1
#         )
        
#         # Save the training history to JSON
#         iem.Helper_Functions.save_history_to_json(history, metadata=f"epochunit_{self.current_epoch_unit+1}", save_path=self.trial_folder)
        
#         # Plot and save the training and validation curve
#         fig = iem.Helper_Functions.plot_train_val_curve(history, training_target_variable="loss")  # Modify target variable as needed.
#         plot_path = os.path.join(self.trial_folder, f"train_val_curve_epochunit_{self.current_epoch_unit+1}.jpg")
#         fig.savefig(plot_path)
#         plt.close(fig)
        
#         self.current_epoch_unit += 1
#         self.save_state(checkpoint_path)
    
#     def run_trial(self):
#         """
#         Runs the trial by loading the dataset, initializing (or resuming) the model,
#         and training for 10 epoch units.
#         """
#         self.load_dataset()
#         if self.model is None:
#             self.model = self.model_builder()
#         self.resume_progress()
#         while self.current_epoch_unit < self.total_epoch_units:
#             print(f"Trial {self.trial_config['Trial']}: Starting epoch unit {self.current_epoch_unit+1}/{self.total_epoch_units}")
#             self.train_epoch_unit()
#         print(f"Trial {self.trial_config['Trial']} completed.")

# def run_trials_from_csv(csv_path):
#     """
#     Reads the CSV file containing trial configurations and runs the experiment for each trial.
    
#     CSV file is expected to have the following columns:
#       Trial, Smoothing, Background, Early Rewarding, Size, Style, Dilation, Attention
#     """
#     df = pd.read_csv(csv_path)
    
#     # Define a root directory to hold all trial folders.
#     experiment_root = "ExperimentResults"
#     os.makedirs(experiment_root, exist_ok=True)
    
#     # Loop over each row (trial) and run the experiment.
#     for idx, row in df.iterrows():
#         trial_config = row.to_dict()
#         print(f"Starting trial: {trial_config.get('Trial', 'Unknown')}")
#         trial_handler = ExperimentHandler(trial_config, experiment_root=experiment_root)
#         trial_handler.run_trial()

# if __name__ == "__main__":
#     # Update this with the path to your CSV file containing trial configurations.
#     csv_path = "trial_config.csv"
#     run_trials_from_csv(csv_path)

┌────────────────────────────────────────────────────────────────────────┐
│  User/CLI                                                             │
│  $ python experiment_framework.py spec.csv EXP_NAME                  │
└────────────┬──────────────────────────────────────────────────────────┘
             │
             ▼
┌────────────────────────────────────────────────────────────────────────┐
│  ExperimentHandler(csv_path, experiment_name, …)                      │
│  ├─ reads CSV into DataFrame                                          │
│  └─ creates   <MODEL_DIR>/<EXP_NAME>/                                 │
└────────────┬──────────────────────────────────────────────────────────┘
             │
             ▼
┌────────────────────────────────────────────────────────────────────────┐
│  run()                                                                │
│  loop rows ▸ trial_row                                                │
│     │                                                                 │
│     ├─ _prepare_trial_folder()    →  …/Trial_XX/ (+ checkpoints/)     │
│     ├─ discover latest checkpoint ─┐                                  │
│     │                             └─▶  models.load_model()   │
│     └─ if none → _build_model()   →  create_modular_dcnn_model()      │
│                                                                       │
│     loop epoch-unit 0-9                                               │
│        ├─ _load_datasets()        →  Data_Functions.prep_dataset…     │
│        ├─ _make_callbacks()       →  {ModelCheckpoint, LR, TB…}       │
│        └─ model.fit(epochs=1)     →  train 1 *epoch-unit*             │
│               └─ Helper_Functions.save_history_to_json() (overwrite)  │
│                                                                       │
│     after 10 units                                                    │
│        └─ _finalise_trial()                                           │
│            ├─ plot_train_val_curve() → Trial_XX/train_val_curve.jpg   │
│            └─ model.save("final_model.keras")                         │
└────────────────────────────────────────────────────────────────────────┘


In [ ]:
# experiment_framework.py
"""
High-level experiment orchestration for the Intron-Exon project.

* 1 CSV == 1 experiment folder ⤏ each row is a *trial* that runs **10 epoch-units**.
* Handles automatic folder creation, checkpointing, history/curve saving, and resume logic.
* Designed to work with the existing IEModules package (Data_Functions, Custom_Models, etc.).

The implementation purposefully leaves a handful of 🔲 *fill-in-the-blank* areas where deeper
project knowledge is needed.  Search for the keyword `TODO` to complete the details.
"""

from __future__ import annotations

import os
import re
import json
import glob
import shutil
from pathlib import Path
from typing import List, Dict, Optional, Tuple

import pandas as pd
import tensorflow as tf

# ── In-house modules ───────────────────────────────────────────────────────────
from IEModules import (
    Data_Functions,
    Custom_Models,
    Custom_Callbacks,
    Custom_Losses,
    Helper_Functions,
)
from IEModules.config import DATA_DIR, MODEL_DIR, LOG_DIR, EPOCH_UNIT_SIZE, EPOCH_UNITS_PER_TRIAL, DEFAULT_BATCH_SIZE, STEPS_PER_EPOCH_UNIT, METRICS

# # ── Constants ──────────────────────────────────────────────────────────────────
# EPOCH_UNIT_SIZE = int(1182630/5)
# EPOCH_UNITS_PER_TRIAL = 10          # <- Each trial always runs exactly 10 units
# DEFAULT_BATCH_SIZE = 28
# STEPS_PER_EPOCH_UNIT = int(EPOCH_UNIT_SIZE/DEFAULT_BATCH_SIZE) # Becomes steps_per_epoch


# Dataset naming helpers -------------------------------------------------------

def build_dataset_filepath(
    split: str,              # 'train' | 'val' | 'test'
    units: int,              # 1-10  (will be zero-padded to 02)
    style_flag: str,         # 'M' or 'I'
    smoothing_flag: str,     # 'C' or 'B'
    base_data_dir: str | Path = DATA_DIR # + Experiment XX/ ############ NEED TO MAKE THIS PART OF CONFIG ##############
) -> str:
    """
    Return the *full* path to the TFRecord whose filename starts with
    '{split}_{units:02d}_{style_flag}_{smoothing_flag}' (case-insensitive).

    Raises:
        FileNotFoundError - no matching file in base_data_dir
        RuntimeError      - more than one match (ambiguous)
    """
    split   = split.lower()
    prefix  = f"{split}_{units:02d}_{style_flag.upper()}_{smoothing_flag.upper()}"
    pattern = f"{prefix}*IEData.tfrecord.gz"        # wildcard swallows shift part
    matches = glob.glob(str(Path(base_data_dir) / pattern), recursive=False)

    if not matches:
        raise FileNotFoundError(f"No TFRecord found for prefix '{prefix}' in {base_data_dir}")
    if len(matches) > 1:
        raise RuntimeError(f"Ambiguous prefix '{prefix}': {matches}")

    return matches[0]   # the unique match



# ── Utility functions ─────────────────────────────────────────────────────────

def latest_checkpoint(ckpt_dir: Path) -> Optional[Path]:
    """Return the most recent .keras checkpoint inside *ckpt_dir* (None if absent)."""
    checkpoints = sorted(ckpt_dir.glob("*.keras"))
    return checkpoints[-1] if checkpoints else None


def parse_epoch_from_ckpt(ckpt_path: Path) -> int:
    """Extract integer epoch-unit from a filename like 'epoch-05-val_metric-0.1234.keras'."""
    m = re.search(r"epoch[-_](\d+)", ckpt_path.name)
    return int(m.group(1)) if m else 0


# ── Main class ────────────────────────────────────────────────────────────────

class ExperimentHandler:
    """Drive the end-to-end training process described by a CSV file."""

    def __init__(
        self,
        csv_path: Path | str, # The csv with trial parameters
        experiment_name: str,
        model_dir: Path | str = MODEL_DIR,
        data_subfolder: Path | str = "",  ######### PROBABLY NEED TO MAKE COMPATIBLE WITH CONFIG ###########
        resume: bool = True,
        batch_size: int = DEFAULT_BATCH_SIZE,
    ) -> None:
        self.csv_path      = Path(csv_path)
        self.experiment    = experiment_name
        self.batch_size    = batch_size
        self.resume_flag   = resume
        self.experiment_dir = Path(model_dir) / experiment_name
        self.experiment_dir.mkdir(parents=True, exist_ok=True)
        self.trials_df = pd.read_csv(self.csv_path)
        # Normalise column names just in case
        self.trials_df.columns = [c.strip() for c in self.trials_df.columns]
        self.base_data_dir = DATA_DIR + data_subfolder

    # ── Public API ──────────────────────────────────────────────────────────

    def run(self) -> None:
        """Loop over trials and epoch-units, honouring resume logic."""
        for row_idx, trial_row in self.trials_df.iterrows():
            trial_id  = int(trial_row["Trial"])  # e.g. 1 ➜ folder Trial_01
            trial_dir = self._prepare_trial_folder(trial_id)

            # ── Determine starting point ---------------------------------------------------
            ckpt_dir = trial_dir / "checkpoints"
            ckpt_dir.mkdir(exist_ok=True)
            start_unit = 0
            model = None
            if self.resume_flag:
                latest_ckpt = latest_checkpoint(ckpt_dir)
                if latest_ckpt is not None:
                    start_unit = parse_epoch_from_ckpt(latest_ckpt) + 1
                    model = models.load_model(latest_ckpt, compile=False)

            # ── Build model from scratch if needed ----------------------------------------
            if model is None:
                model = self._build_model(trial_row)

            # ── Training loop over *epoch units* -----------------------------------------
            for unit in range(start_unit, EPOCH_UNITS_PER_TRIAL):
                print(f"\n⚙️  Trial {trial_id:02d}  •  Epoch-Unit {unit+1}/{EPOCH_UNITS_PER_TRIAL}")

                train_ds, val_ds = self._load_datasets(trial_row)
                
                '''#################### GOTTA UPDATE THIS FOR CONFIG ##############'''
                callbacks = self._make_callbacks(trial_row, ckpt_dir, unit)

                
                history = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    epochs=EPOCH_UNITS_PER_TRIAL,
                    steps_per_epoch=STEPS_PER_EPOCH_UNIT,
                    callbacks=callbacks,
                )

                # Overwrite history JSON each unit (requirement)
                Helper_Functions.save_history_to_json(
                    history,
                    metadata="history",  # single file always named history_*.json
                    save_path=str(trial_dir),
                )

            # ── End-of-trial artifacts ----------------------------------------------------
            self._finalise_trial(model, trial_dir)

        print("\n✅ All trials complete!")

    # ── Internals ───────────────────────────────────────────────────────────

    def _prepare_trial_folder(self, trial_num: int) -> Path:
        trial_dir = self.experiment_dir / f"Trial_{trial_num:02d}"
        trial_dir.mkdir(parents=True, exist_ok=True)
        (trial_dir / "checkpoints").mkdir(exist_ok=True)
        return trial_dir

    
    def _dataset_path_from_row(self, row: pd.Series, split: str, base_data_dir: str) -> str:
        """
        Given a CSV row and a split ('train', 'val', 'test'),
        return the full path to the TFRecord.
        """
        units_flag     = int(row["Size"])                       # 1-10
        style_flag     = str(row["Style"]).upper()[0]           # 'M' | 'I'
        smoothing_flag = (
            "C" if str(row["Smoothing"]).upper().startswith("C") else "B"
        )

        # shift descriptor intentionally ignored
        return build_dataset_filepath(
            split          = split,
            units          = units_flag,
            style_flag     = style_flag,
            smoothing_flag = smoothing_flag,
            base_data_dir  = base_data_dir
        )

    def _load_datasets(self, row: pd.Series, test: bool = False):
        """
        Instantiate *separate* train / validation datasets based on the CSV row.
        Also works as a train-test-set loader, set test = True when using _load_datasets
        test dataset pretends to be the val set when test = True
        """
        train_path = self._dataset_path_from_row(row, split="train")
        if test:
            val_path = self._dataset_path_from_row(row, split="test")
        else:
            val_path = self._dataset_path_from_row(row, split="val")

        train_ds = Data_Functions.prep_dataset_from_tfrecord(
            train_path,
            batch_size   = self.batch_size,
            shuffled     = True,                       # shuffle only training
            cut_background = not bool(row["Background"]),
        )
        train_ds = train_ds.repeat()

        val_ds = Data_Functions.prep_dataset_from_tfrecord(
            val_path,
            batch_size   = self.batch_size,
            shuffled     = False,                      # never shuffle validation
            cut_background = not bool(row["Background"]),
        )
        return train_ds, val_ds

    def _build_model(self, row: pd.Series) -> Model:
        """Create and compile a new model according to hyper-params in *row*."""
        # 🔲 TODO - Make compatible with config
        dilation_mult = float(row.get("Dilation", 1.0))
        use_attention = bool(row.get("Attention", False))

        model = Custom_Models.create_modular_dcnn_model(
            dilation_multiplier=dilation_mult,
            use_local_attention=use_attention,
            use_long_range_attention=use_attention,
            use_final_attention=use_attention,
        )

        loss_fn = self._select_loss(row)
        metrics = METRICS
        model.compile(
            optimizer=optimizers.Adam(),
            loss=loss_fn,
            metrics=metrics,
        )
        return model

    def _select_loss(self, row: pd.Series):
        """Return the correct loss function instance (handles Early Rewarding switch)."""
        early_rewarding = bool(row["Early Rewarding"]) # True or False
        smoothing = str(row["Smoothing"]) # None (as a string), Custom, or Proper
        background = bool(row["Background"]) # True (Included) or False (Removed)
        # Construct a config for loss function params
        # pull that in
        # update the others based on the three variables read in above
        # choose loss fn with hyper params
        chosen_loss = "FILL IN CHOSEN LOSS STUFF"
        return chosen_loss

# ########### NEED TO MAKE THIS COMPATIBLE WITH CONFIG ################
    def _make_callbacks(
        self,
        row: pd.Series,
        ckpt_dir: Path,
        epoch_unit_idx: int,
    ) -> List[callbacks.Callback]:
        """Create per-epoch-unit callback list."""
        ckpt_cb = callbacks.ModelCheckpoint(
            filepath=str(ckpt_dir / f"epoch-{epoch_unit_idx:03d}.keras"),
            save_weights_only=False,
            save_best_only=False,
        )

        reduce_lr_cb = Custom_Callbacks.StatefulReduceLROnPlateau(
            monitor="val_loss",             # 🔲 or val_no_background_f1
            factor=0.5,
            patience=5,
            verbose=1,
            state_save_filepath=str(ckpt_dir / "lr_state.json"),
        )

        tensorboard_dir = LOG_DIR + f"/{self.experiment}/trial_{row['Trial']:02d}/unit_{epoch_unit_idx:02d}"
        tb_cb = callbacks.TensorBoard(log_dir=tensorboard_dir, histogram_freq=1)

        cleanup_cb = Custom_Callbacks.CleanupCallback()
        return [ckpt_cb, reduce_lr_cb, tb_cb, cleanup_cb]

    def _finalise_trial(self, model: Model, trial_dir: Path) -> None:
        """Plot train/val curves & save a compact *production* checkpoint."""
        # 🔲 TODO - capture the accumulated History over 10 units (requires persistence).
        history_json = trial_dir + "history_training_history.json"
        if history_json.exists():
            with open(history_json) as fh:
                history_dict = json.load(fh)
            # Use helper to generate plot and save
            fig = Helper_Functions.plot_train_val_curve(history_object=type("H", (), {"history": history_dict})(),
                                                        training_target_variable="loss")
            fig.savefig(trial_dir + "train_val_curve.jpg")

        # Save final model (weights only)
        model.save(trial_dir + "final_model.keras")


# ── Command-line helper ───────────────────────────────────────────────────────
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description="Run an Intron-Exon experiment from a CSV spec.")
    parser.add_argument("csv", help="Path to experiment CSV file")
    parser.add_argument("name", help="Experiment folder name (under Models/)")
    parser.add_argument("--no-resume", action="store_true", help="Start from scratch, ignore existing checkpoints")
    parser.add_argument("--batch", type=int, default=DEFAULT_BATCH_SIZE, help="Batch size")
    args = parser.parse_args()

    handler = ExperimentHandler(
        csv_path=args.csv,
        experiment_name=args.name,
        resume=not args.no_resume,
        batch_size=args.batch,
    )
    handler.run()

┌────────────────────────────────────────────────────────────────────────┐
│  External call                                                        │
│  best_hp, tuner = run_tuning(data_spec, project_name="exp01")         │
└────────────┬──────────────────────────────────────────────────────────┘
             │
             ▼
┌────────────────────────────────────────────────────────────────────────┐
│  run_tuning()                                                         │
│  ├─ _prepare_datasets()         →  Data_Functions.prep_dataset…       │
│  │                                returns (train_ds, val_ds)          │
│  ├─ DCNNHyperModel(data_spec)                                         │
│  │    │                                                               │
│  │    └─ build(hp)                                                    │
│  │         ├─ hp.* search-space (lr, dilation_mult, attention …)      │
│  │         ├─ Custom_Models.create_modular_dcnn_model()               │
│  │         └─ model.compile( CustomBinaryFocalLoss , metrics )        │
│  ├─ kt.Hyperband( … directory=Models/<proj>/tuner/ )                  │
│  ├─ tuner.search(train_ds, val_ds, callbacks=[EarlyStopping])         │
│  ├─ best_hp = tuner.get_best_hyperparameters(1)[0]                    │
│  └─ _save_best(best_hp, project_name) → best_hparams.json             │
└────────────────────────────────────────────────────────────────────────┘

   ▲ load_best_hparams(project_name)  ←────────────┘ (JSON helper)


In [ ]:
# Hyperparameter_Tuning.py
"""
Keras-Tuner wrapper for the Intron-Exon project.

Usage
-----
>>> from IEModules import Hyperparameter_Tuning as hpt
>>> best_hp, tuner = hpt.run_tuning(
...     data_spec=row_from_csv,
...     project_name="exp01_hyperband"
... )
"""

from __future__ import annotations
import json, os, datetime
from pathlib import Path
from typing import Dict, Tuple

import keras_tuner as kt
import tensorflow as tf

from . import (
    Custom_Models,
    Custom_Losses,
    Data_Functions,
    Custom_Metrics,
    config
)

# ──────────────────────────────────────────────────────────────────────────────
# 1. HyperModel definition
# ──────────────────────────────────────────────────────────────────────────────
class DCNNHyperModel(kt.HyperModel):
    """Wraps create_modular_dcnn_model with a tunable search-space."""
    def __init__(self, data_spec: Dict):
        self.data_spec = data_spec          # row from your experiment CSV

    # --- mandatory -----------------------------------------------------------
    def build(self, hp: kt.HyperParameters) -> Model:
        # Search-space examples — extend freely.
        dilation_mult = hp.Float("dilation_mult", 0.5, 2.0, step=0.25)
        use_attention = hp.Boolean("attention", default=True)
        lr            = hp.Float("lr", 1e-4, 3e-3, sampling="log")

        model = Custom_Models.create_modular_dcnn_model(
            dilation_multiplier=dilation_mult,
            use_local_attention=use_attention,
            use_long_range_attention=use_attention,
            use_final_attention=use_attention,
        )

        model.compile(
            optimizer=optimizers.Adam(learning_rate=lr),
            loss      = Custom_Losses.CustomBinaryFocalLoss(),
            metrics   = [Custom_Metrics.CustomNoBackgroundF1Score(num_classes=5)],
        )
        return model

    # --- optional ------------------------------------------------------------
    def fit(self, hp, model, *args, **kwargs):
        """You can override to inject custom callbacks—or just omit."""
        return super().fit(hp, model, *args, **kwargs)

# ──────────────────────────────────────────────────────────────────────────────
# 2. Helper: get train/val datasets exactly once per tuner process
# ──────────────────────────────────────────────────────────────────────────────
def _prepare_datasets(data_spec: Dict, batch_size=32):
    tfrecord_path = Path(config.DATA_DIR) / "YOUR_DEFAULT.tfrecord.gz"
    # TODO - replace -- use data_spec to choose correct file
    ds = Data_Functions.prep_dataset_from_tfrecord(
        tfrecord_path,
        batch_size=batch_size,
        shuffled=True,
        cut_background=not bool(data_spec["Background"])
    )
    # simplistic 80-20 split
    n = ds.cardinality().numpy()
    train_ds = ds.take(int(n*0.8))
    val_ds   = ds.skip(int(n*0.8))
    return train_ds, val_ds

# ──────────────────────────────────────────────────────────────────────────────
# 3. Public API
# ──────────────────────────────────────────────────────────────────────────────
def run_tuning(
    data_spec: Dict,
    project_name: str,
    max_trials: int = 30,
    overwrite: bool = False,
) -> Tuple[kt.HyperParameters, kt.Tuner]:

    # Materialise datasets *once* so every trial re-uses the same TF graph
    train_ds, val_ds = _prepare_datasets(data_spec)

    hypermodel = DCNNHyperModel(data_spec)

    tuner = kt.Hyperband(
        hypermodel,
        objective   = kt.Objective("val_no_background_f1", direction="max"),
        max_epochs  = 15,
        factor      = 3,
        directory   = str(Path(config.MODEL_DIR) / project_name),
        project_name= "tuner",
        overwrite   = overwrite
    )

    stop_early = callbacks.EarlyStopping(
        monitor="val_no_background_f1",
        mode="max",
        patience=5,
        restore_best_weights=True
    )

    tuner.search(
        train_ds,
        validation_data = val_ds,
        callbacks       = [stop_early],
    )

    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    _save_best(best_hp, project_name)

    return best_hp, tuner

# ──────────────────────────────────────────────────────────────────────────────
# 4. Convenience helpers
# ──────────────────────────────────────────────────────────────────────────────
def _save_best(best_hp: kt.HyperParameters, project_name: str):
    out = Path(config.MODEL_DIR) / project_name / "best_hparams.json"
    with open(out, "w") as fh:
        json.dump(best_hp.values, fh, indent=2)
    print(f"Best HP written to {out}")

def load_best_hparams(project_name: str) -> Dict:
    path = Path(config.MODEL_DIR) / project_name / "best_hparams.json"
    with open(path) as fh:
        return json.load(fh)


In [ ]:
# anywhere - CLI, notebook, or even inside ExperimentHandler
from IEModules.Hyperparameter_Tuning import run_tuning, load_best_hparams

best_hp, tuner = run_tuning(
    data_spec   = row_from_csv_as_dict,
    project_name= "exp01_hyperband"
)
print("Best LR  :", best_hp['lr'])
print("Dilation :", best_hp['dilation_mult'])
